### Imports and Constants

In [4]:
import torch
import torchvision.models as models

from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms



import pandas

import kagglehub

import matplotlib.pyplot as plt

In [24]:
# Constants

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
COLOR_CHANNELS = 3
EPOCHS = 12
LEARNING_RATES = [0.0001, 0.001, 0.01, 0.1]
ACCURACY_THRESHOLD = 0.2

# GPU acceleration

print(torch.__version__)
print("MPS built:", torch.backends.mps.is_built())
print("MPS available:", torch.backends.mps.is_available())

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Using device:", device)

2.6.0
MPS built: True
MPS available: True
Using device: mps


### Pretrained Model - resnet101
reference:

https://arxiv.org/abs/1512.03385

https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html


In [25]:
import torch
import torchvision.models as models
                                                                                         
resnet101 = models.resnet101(weights=True)          # load resnet model 

resnet_layers = resnet101.children()                # split resnet into array of layers 
resnet_layers = list(resnet_layers)[:-1]            # remove final layer from the array

    
resnet101 = torch.nn.Sequential(*resnet_layers).to(device)     # recombine modified layers, move to gpu if available
   
resnet101.eval()                                     # Set model to evaluation mode



Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


### Custom Dataset Class - ArtworkDataset

reference:

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [26]:
from sklearn.preprocessing import MinMaxScaler
# Create a scaler object
scaler = MinMaxScaler()

In [27]:
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import Dataset, DataLoader


class ArtworkDataset(Dataset):

    
    def __init__(self, artworks_frame, artwork_images_path, transform=None):
        
            # remove USD and whitespace from end of price strings 
            # convert price strings to floats
        artworks_frame['price'] = artworks_frame['price'].str.replace('USD', '', regex=False).str.strip().astype(float)

        artworks_frame = artworks_frame[['price']] # trim dataframe to only hold price column

        #print(type(artworks_frame))
        #print(artworks_frame)
        

        scaled_prices = scaler.fit_transform(artworks_frame) # array of [0, 1] normalized prices

        #print(type(artworks_frame))
        #artworks_frame = pd.DataFrame(artworks_frame, columns=['price']) # convert back to dataframe

        artworks_frame['price'] = scaled_prices # set prices to scaled prices within dataframe

  

        

            # add dataframe, image path, and transform to dataset

        self.artworks_frame      = artworks_frame
        self.artwork_images_path = artwork_images_path
        self.transform           = transform

    
    def __len__(self):
        return len(self.artworks_frame)

    
    def __getitem__(self, idx):     
                                                                # Get image path
        image_path = self.artwork_images_path + "/" + "image_" + str(idx + 1) + ".png" # assume index + 1 is same as image id

          
        image = read_image(image_path, mode=ImageReadMode.RGB).float() # Get image as tensor with only RGB channels
        price = self.artworks_frame.iloc[idx]['price']                 # Get price

        if self.transform:                                     # Apply transformation to image
            image  = self.transform(image)
            
        return image, torch.tensor(price, dtype=torch.float32) # Return transformed image and Price tensor


### Dataset Download - art-price-dataset

In [28]:
    import kagglehub
    
    print( "DATASET DOWNLOADED.   path:   " + kagglehub.dataset_download("flkuhm/art-price-dataset") )

DATASET DOWNLOADED.   path:   /Users/jackblackburn/.cache/kagglehub/datasets/flkuhm/art-price-dataset/versions/1


### Data Preprocessing - get_art_price_dataset(), get_image_transform()


In [29]:
def get_art_price_dataset_path():
    
                            # ensure dataset is downloaded & get path
    import kagglehub
    
    dataset_path = kagglehub.dataset_download("flkuhm/art-price-dataset")
    
    
    print( "DATASET FOUND.   path:   " + kagglehub.dataset_download("flkuhm/art-price-dataset") )
    
                                            # get csv file and image directory paths
    
    csv_path      = dataset_path + "/" + "artDataset.csv" # csv file of data about artworks
    artworks_path = dataset_path + "/" + "artDataset"     # directory of artwork images

    return csv_path, artworks_path


In [30]:
def get_image_transform():

    import torchvision.transforms as transforms
    
    # Define a data transformation for image preprocessing
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image data
    ])

    return transform


### Multi-Layer Perceptron Class - Net, train(), validate()

In [31]:
import torch

class Net(torch.nn.Module):                     # Define Multi-Layer Perceptron Class
    
    def __init__(self, input_size, n_hidden_nodes):     # INIT

        super(Net, self).__init__()                                 # Super
        
                                                                    # Set NN Properties
        self.n_hidden_nodes = n_hidden_nodes                        # Number of hidden nodes in each layer
        self.fc1 = torch.nn.Linear(input_size, n_hidden_nodes)      # Fully connected layer 1
        self.fc2 = torch.nn.Linear(n_hidden_nodes, n_hidden_nodes)  # Fully connected layer 2
        self.fc3 = torch.nn.Linear(n_hidden_nodes, n_hidden_nodes)  # Fully connected layer 3
        self.out = torch.nn.Linear(n_hidden_nodes, 1)               # Output layer



    
    
    def forward(self, x):                         # FORWARD
  

        sigmoid = torch.nn.Sigmoid()
        
        x = sigmoid(self.fc1(x))        # Apply sigmoid activation to the first hidden layer
        x = sigmoid(self.fc2(x))        # Apply sigmoid activation to the second hidden layer
        x = sigmoid(self.fc3(x))        # Apply sigmoid activation to the third hidden layer

                  
        x = sigmoid(self.out(x))        # Get output and squash between [0, 1] via sigmoid activation

        
        return x

In [36]:
def train(epoch, model, train_loader, optimizer):

    print("TRAINING MODEL")
       
    model.train()  # Set the model in training mode

    
    
        # Iterate over batches in the training loader

    
    
    for images, prices in train_loader:

        images = images.to(device)
        prices = prices.to(device)

        
             # Extract features & flatten into vector
        features = resnet101(images) 
        features = features.view(features.size(0), -1) 
        
        
           
        optimizer.zero_grad()     # Clear the gradients from the previous iteration

        outputs = model(features)  # Forward pass: compute the model's outputs

        
            # Use mean squared error to calculate loss
        
        loss = torch.nn.functional.mse_loss(outputs.squeeze(), prices) 
        
        loss.backward()           # Backpropagate the gradients
        optimizer.step()          # Update the model's parameters using the computed gradients


In [40]:
def validate(loss_vector, accuracy_vector, model, validation_loader):

    print("VALIDATING MODEL")
    
    model.eval()              # Set the model in evaluation mode
    
    val_loss, correct = 0, 0  # Initialize variables for loss and correct predictions



        # Iterate over batches in the validation loader
    
    for images, prices in validation_loader:

        images = images.to(device)
        prices = prices.to(device)
        

             # Extract features & flatten into vector
        
        features = resnet101(images) 
        features = features.view(features.size(0), -1) 
        
        outputs = model(features)  # Forward pass: compute the model's predictions
        
        val_loss += torch.nn.functional.mse_loss(outputs.squeeze(), prices).data  # Compute the mean squared loss

        
       

            # Compute number of "correct" predictions (within 10% of target)
        
        deviation           = torch.abs(outputs.squeeze() - prices) / prices  # list of relative deviations from target
        correct_predictions = (deviation <= ACCURACY_THRESHOLD).sum().item()  # number of predictions with deviation < threshold
    
    
            # Update the count of correct predictions
        
        correct += correct_predictions



        # Calculate the accuracy as a percentage 

    accuracy = 100. * correct / len(validation_loader.dataset)  

        # Append the accuracy to a list

    accuracy_vector.append(accuracy)  

    val_loss /= len(validation_loader)  # Calculate the average validation loss
    
    loss_vector.append(val_loss)        # Append the validation loss to a list


    

        # Print the validation results
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))



### Main

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split

def main():

    
    print('Using PyTorch version:', torch.__version__)
    
        # Load the dataset
    
    csv_path, artworks_path = get_art_price_dataset_path()
    data = pd.read_csv(csv_path)
    
    
    
        # split dataset into training / validation sets
    
    train_data, validation_data = train_test_split(data, test_size=0.3, random_state=42)
    
        # build dataset loaders
    
    transform           = get_image_transform()
    
    train_dataset       = ArtworkDataset(train_data, artworks_path, transform=transform)
    validation_dataset  = ArtworkDataset(validation_data,  artworks_path, transform=transform)
    
    train_loader        = DataLoader(train_dataset, batch_size=4, shuffle=True)
    validation_loader   = DataLoader(validation_dataset, batch_size=4, shuffle=False)
    
    
    
        # create mlp
    
    mlp = Net(input_size = 2048, n_hidden_nodes = 512).to(device)
    
        # Define the optimizer for training (Stochastic Gradient Descent)
    
    optimizer = torch.optim.SGD(mlp.parameters(), lr=LEARNING_RATES[2])  # Learning rate specified elsewhere
    
    loss_vector = []  # List to store training loss values
    acc_vector = []   # List to store training accuracy values
    
    
    for epoch in range(1, EPOCHS + 1):
        
        print('Epoch {}'.format(epoch))
        
            # Train the model on the training dataset
        train(epoch, mlp, train_loader, optimizer)
        
            # Validate the model on the validation dataset and collect loss and accuracy data
        validate(loss_vector, acc_vector, mlp, validation_loader)
            




# Run

In [42]:
main()

Using PyTorch version: 2.6.0
DATASET FOUND.   path:   /Users/jackblackburn/.cache/kagglehub/datasets/flkuhm/art-price-dataset/versions/1
Epoch 1
TRAINING MODEL


/var/folders/2l/rz8ydl4561x3wb2q0wms5w540000gn/T/ipykernel_39154/4280577537.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artworks_frame['price'] = scaled_prices # set prices to scaled prices within dataframe
/var/folders/2l/rz8ydl4561x3wb2q0wms5w540000gn/T/ipykernel_39154/4280577537.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artworks_frame['price'] = scaled_prices # set prices to scaled prices within dataframe


VALIDATING MODEL

Validation set: Average loss: 0.0951, Accuracy: 0/227 (0%)

Epoch 2
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.1002, Accuracy: 0/227 (0%)

Epoch 3
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.0939, Accuracy: 0/227 (0%)

Epoch 4
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.0950, Accuracy: 0/227 (0%)

Epoch 5
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.0966, Accuracy: 0/227 (0%)

Epoch 6
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.0955, Accuracy: 0/227 (0%)

Epoch 7
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.0928, Accuracy: 0/227 (0%)

Epoch 8
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.1052, Accuracy: 0/227 (0%)

Epoch 9
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.0927, Accuracy: 1/227 (0%)

Epoch 10
TRAINING MODEL
VALIDATING MODEL

Validation set: Average loss: 0.0963, Accuracy: 0/227 (0%)

Epoch 11
TRA